In [2]:
#导入各种包
import numpy as np
import pandas as pd
import time
import csv
from tqdm import tqdm
import shutil
import gzip

In [6]:
#语料生成模块,输入参数：读取的csv，输出的文件名，start为开始条数，end为结束条数，need_all为True则读取所有数据，否则读取指定数据
#注：0为query_id,1为query,2为title_id,3为title,4为label。
#已经做了优化，内存占用极低，效率很高（pandas内存占用较大，没有采用）
def make_csv_from_csv(filename,outputfile,start,end,need_all):
    with open(outputfile, 'w',newline = '') as output:
        with open(filename) as csv_file:
            #处理开始时间
            start_time = time.time()
            csv_reader = csv.reader(csv_file, delimiter=',')
            #python内置csv.writer
            writer = csv.writer(output)
            i = 0
            if need_all==True:
                for row in tqdm(csv_reader,mininterval=1.0):
                    if (i >=start and i<end):
                        writer.writerow(row)
                    i+=1
            else:
                ttype = input('输入需要的列：(逗号分隔)')
                start_time = time.time()
                ttype_list = list(map(int,ttype.split(',')))
                for row in tqdm(csv_reader,mininterval=1.0):
                    if (i >=start and i<end):
                        line = []
                        for j in ttype_list:
                            line.append(row[j])
                        writer.writerow(line)
                    i+=1
                        
                    
                    
            
            print("make text from "+filename+" using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
make_csv_from_csv('train.csv','test.csv',start = 17000,end = 20000, need_all = False)

输入需要的列：(逗号分隔)0,2,4


20000it [00:00, 466220.27it/s]


make text from train.csv using time 0.04484200477600098 seconds


True

In [9]:
#语料生成模块,输入参数：读取的csv，输出的文件名，ttype为指定输出。0为query_id,1为query,2为title_id,3为title,4为label。
#已经做了优化，内存占用极低，效率很高
def make_text_from_csv(filename,outputfile,ttype):
    with open(outputfile, 'w') as output:
        with open(filename) as csv_file:
            #处理开始时间
            start_time = time.time()
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in tqdm(csv_reader,mininterval=1.0):
                need = row[ttype]
                output.write("{0}\n".format(need))
                
            print("make text from "+filename+" using time "+str(time.time()-start_time)+" seconds")
    return True
#调用该函数
make_text_from_csv('train.csv','title.txt',3)

20000it [00:00, 142991.70it/s]


make text from train.csv using time 0.1453719139099121 seconds


True

In [27]:
#将train.csv与test.csv的所有内容写到一个txt文件中
def make_txt_from_2csv(outputfile):
    with open(outputfile, 'w') as output:
        #写train.csv
        with open("train.csv") as csv_file:
            #处理开始时间
            start_time = time.time()
            csv_reader = csv.reader(csv_file, delimiter=',')
            line_count = 0
            for row in tqdm(csv_reader,mininterval=1.0):
                query_id = row[0]
                query = row[1]
                title_id = row[2]
                title = row[3]
                label = row[4]
                output.write("{0},{1},{2},{3}\n".format(query_id,query,title_id, title,label))
        print("make corpus from train.csv, using time "+str(time.time()-start_time)+" seconds")
        
        #写test.csv
        with open("test.csv") as csv_file:
            #处理开始时间
            start_time = time.time()
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in tqdm(csv_reader,mininterval=1.0):
                query_id = row[0]
                query = row[1]
                title_id = row[2]
                title = row[3]
                label = row[4]
                output.write("{0},{1},{2},{3}\n".format(query_id,query,title_id, title,label))
        print("make corpus from test.csv, using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
make_txt_from_2csv('a.txt')

20000it [00:00, 156489.57it/s]


make corpus from train.csv with 0lines, using time 0.1442575454711914 seconds


3000it [00:00, 155949.14it/s]


make corpus from test.csv with 0lines, using time 0.023122549057006836 seconds


True

In [21]:
#文本组合模块，输入一个txt文件的列表（四个），传入总长度，输出一个合成的文件
#已经做了优化，内存占用极低，效率很高
def text_conbine(feature_file_list,outputfile,length):
    with open(outputfile,'w+') as output:
        #处理开始时间
        start_time = time.time()
        with open(feature_file_list[0],'r') as f0:
            with open(feature_file_list[1],'r') as f1:
                with open(feature_file_list[2],'r') as f2:
                    with open(feature_file_list[3],'r') as f3:
                        #tqdm可以提供进度条，监控总长度变化
                        for i in tqdm(range(length),mininterval=1.0):
                            #读取一行数据，并且去除换行符
                            line = f0.readline().strip('\n') + ','+ f1.readline().strip('\n')+ ','+ f2.readline().strip('\n')+ ','+ f3.readline().strip('\n')
                            output.write("{0}\n".format(line))      

                    
            print(" using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
text_conbine(['query_repeat.txt','title_repeat.txt','train_len_feature','tfidf_common_feature.txt'],'peter_feature.txt',length=20000)

100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 257095.29it/s]


 using time 0.08179426193237305 seconds


In [10]:
#文本组合模块，输入一个txt文件的列表（两个），传入总长度，输出一个合成的文件
#已经做了优化，内存占用极低，效率很高
def feature_conbine(feature_file_list,outputfile,length):
    with open(outputfile,'w+') as output:
        #处理开始时间
        start_time = time.time()
        with open(feature_file_list[0],'r') as f0:
            with open(feature_file_list[1],'r') as f1:
            #tqdm可以提供进度条，监控总长度变化
                for i in tqdm(range(length),mininterval=1.0):
                    #读取一行数据，并且去除换行符
                    line = f0.readline().strip('\n') + ' '+ f1.readline().strip('\n')
                    output.write("{0}\n".format(line))      

                    
            print(" using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
feature_conbine(['query.txt','title.txt'],'query_title.txt',length=20000)

100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 174556.84it/s]


 using time 0.12152719497680664 seconds


True

In [ ]:
#文本组合模块，输入一个txt文件的列表（十一个），传入总长度，输出一个合成的文件
#已经做了优化，内存占用极低，效率很高
def feature_conbine(feature_file_list,outputfile,length):
    with open(outputfile,'w+') as output:
        #处理开始时间
        start_time = time.time()
        with open(feature_file_list[0],'r') as f0:
            with open(feature_file_list[1],'r') as f1:
                with open(feature_file_list[2],'r') as f2:
                    with open(feature_file_list[3],'r') as f3:
                        with open(feature_file_list[4],'r') as f4:
                            with open(feature_file_list[5],'r') as f5:
                                with open(feature_file_list[6],'r') as f6:
                                    with open(feature_file_list[7],'r') as f7:
                                        with open(feature_file_list[8],'r') as f8:
                                            with open(feature_file_list[9],'r') as f9:
                                                with open(feature_file_list[10],'r') as f10:
            #tqdm可以提供进度条，监控总长度变化
                                                    for i in tqdm(range(length),mininterval=1.0):
                                                        #读取一行数据，并且去除换行符
                                                        line = f0.readline().strip('\n') + ','+ f1.readline().strip('\n')+ ','+ f2.readline().strip('\n')\
                                                            + ','+ f3.readline().strip('\n')+ ','+ f4.readline().strip('\n')+ ','+ f5.readline().strip('\n')\
                                                            +','+ f6.readline().strip('\n')+','+ f7.readline().strip('\n')+','+ f8.readline().strip('\n')\
                                                            +','+ f9.readline().strip('\n')+','+ f10.readline().strip('\n')
                                                        output.write("{0}\n".format(line))      

                    
            print(" using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
feature_conbine(['train_dis.txt','train_len_feature.txt','train_query_title_dis_v2.txt','train_tfidf_LR.txt','train_tfidf_mh.txt','train_w2v_sim_mat_mean.txt','query_repeat.txt','title_repeat.txt','query_repeat.txt','train_dis_v2.txt','train_query_title_dis_v2.txt','tfidf_common_feature.txt'],'feature_5.0.txt',length=20000)

In [3]:
#文本组合模块，输入一个txt文件的列表（十个），传入总长度，输出一个合成的文件
#已经做了优化，内存占用极低，效率很高
def feature_conbine(feature_file_list,outputfile,length):
    with open(outputfile,'w+') as output:
        #处理开始时间
        start_time = time.time()
        with open(feature_file_list[0],'r') as f0:
            with open(feature_file_list[1],'r') as f1:
                with open(feature_file_list[2],'r') as f2:
                    with open(feature_file_list[3],'r') as f3:
                        with open(feature_file_list[4],'r') as f4:
                            with open(feature_file_list[5],'r') as f5:
                                with open(feature_file_list[6],'r') as f6:
                                    with open(feature_file_list[7],'r') as f7:
                                        with open(feature_file_list[8],'r') as f8:
                                            with open(feature_file_list[9],'r') as f9:
                                                #tqdm可以提供进度条，监控总长度变化
                                                for i in tqdm(range(length),mininterval=1.0):
                                                    #读取一行数据，并且去除换行符
                                                    line = f0.readline().strip('\n') + ','+ f1.readline().strip('\n')+ ','+ f2.readline().strip('\n')\
                                                        + ','+ f3.readline().strip('\n')+ ','+ f4.readline().strip('\n')+ ','+ f5.readline().strip('\n')\
                                                        +','+ f6.readline().strip('\n')+','+ f7.readline().strip('\n')+','+ f8.readline().strip('\n')\
                                                        +','+ f9.readline().strip('\n')
                                                    output.write("{0}\n".format(line))      

                    
            print(" using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
feature_conbine(['len_feature.txt','query_repeat.txt','query_title_dis.txt','tfidf_common_feature.txt','tfidf_LR.txt',
                 'tfidf_MH.txt','title_repeat.txt','title_tfidf_common_feature.txt','w2v_d2v_distance.txt','w2v_sim_mat_mean.txt'],'all_feature.txt',length=20000)

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 85874.06it/s]


 using time 0.24786138534545898 seconds


True

In [30]:
import lightgbm as lgb

#读取存储的lgb模型，传入测试集csv和测试集总特征并且预测结果，保存到文件中
def predict(lgb_model,feature_file,test_file,out_file):
    #读取 
    bst = lgb.Booster(model_file=lgb_model)
    #预测
    result = bst.predict(feature_file)
    #newline=''防止写csv时每两行数据中空一行
    with open(out_file, 'w',newline = '') as output:
        with open(test_file) as csv_file:
            #处理开始时间
            start_time = time.time()
            #打开csv_reader以及csv_writer
            csv_reader = csv.reader(csv_file, delimiter=',')
            writer = csv.writer(output)
            
            line_count = 0
            for row in csv_reader:
                query_id = row[0]
                title_id = row[2]
                pred = result[line_count]
                #写下csv一行
                writer.writerow([query_id,title_id,pred])
                line_count+=1
    print("complete, "+"using time:"+str(time.time()-start_time))

        
#调用该函数
predict('lgb_model.txt','features_v3.2.2.txt','test.csv','result.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [8]:
#语料生成模块,输入参数：读取的txt，输出的文件名，开始和结束位置
#已经做了优化，内存占用极低，效率很高
#从txt中截取需要的行（所有内容）
def make_text_from_text(filename,outputfile,begin,end):
    with open(outputfile, 'w',newline = '') as output:
        with open(filename) as f:
            #处理开始时间
            start_time = time.time()
            for i in tqdm(range(end),mininterval=1.0):
                if i <begin:
                    f.readline().strip('\n')
                    continue
                if i >= end:
                    break
                need = f.readline().strip('\n')
                #print(need)
                output.write("{0}\n".format(need))
                #print(i)
            print("make text from "+filename+" using time "+str(time.time()-start_time)+" seconds")
            
    return True

#调用该函数
make_text_from_text('lb.txt','lb_train.txt', begin=0, end=17000, )

100%|████████████████████████████████████████████████████████████████████████| 17000/17000 [00:00<00:00, 629964.55it/s]


make text from lb.txt using time 0.028925418853759766 seconds


True

In [25]:
#删除text中的指定内容（对每一行进行删除）,运行后用户输入为几，就删除掉几的内容（从0开始）
#已经优化，占用内存极低
def del_text_from_text(filename,outputfile,length):
    start_time = time.time()
    ttype = input('输入需要删除的位数：(逗号分隔)')
    ttype_list = list(map(int,ttype.split(',')))
    #倒序
    ttype_list.sort(reverse=True)
    
    with open(outputfile, 'w') as output:
        with open(filename,'r') as f:
            for i in tqdm(range(length),mininterval=1.0):
                lst = f.readline().strip('\n').split(',')
                
                for j in ttype_list:
                    del lst[j]
                s = ','.join(lst)
                output.write("{0}\n".format(s))
    print("complete, using time "+str(time.time()-start_time)+" seconds")
    return True

#调用该函数
del_text_from_text('lb.txt','test.txt',length = 20000)

输入需要删除的位数：(逗号分隔)0


100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 312796.51it/s]


complete, using time 1.042250156402588 seconds


True

In [19]:

#压缩指定txt，生成txt.gz文件（减少空间占用）（内容变为二进制文件）
def compress_feature(filename):
    #处理开始时间
    start_time = time.time()
    #打开被压缩文件，打开压缩文件（没有则创建）
    with open(filename,'rb') as ff,gzip.open(filename+'.gz','wb') as cf:
        #使用了复制模式，速度很快
        shutil.copyfileobj(ff, cf)
    
    print("压缩文件："+filename+"， using time "+str(time.time()-start_time)+" seconds")        
    return True

#调用该函数
compress_feature('query.txt')

压缩文件：query.txt， using time 0.018685579299926758 seconds


True

In [23]:
#从txt.gz压缩文件中读取所有数据，返回一个含有所有数据的list
def load_list_from_cpfile(cpfile,length):
    with gzip.open(cpfile, 'rb') as cpfile:
        start_time = time.time()
        file_content = cpfile.readlines()
        result = []
        for i in tqdm(range(length),mininterval=1.0):
            #编码为二进制，返回为十进制，并去掉\t\r\n这些限定符
            need = file_content[i].decode().strip('\t\r\n')
            result.append(need)
            
    print("using time "+str(time.time()-start_time)+" seconds")        
    return result
    
a = load_list_from_cpfile('query.txt.gz',20000)
#展现前两个
print(a[:2])

100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 823785.52it/s]


using time 0.05173969268798828 seconds
['1427 5661 29788 1427 387 2299 372 22 1586 1025 218 165 218 27 7092 22 266 25817 4550 2136 11 60847 9156 1077 797 27 689 3447 11146 108 1667 46829 161702 1113017 548478 24274 5062 46829', '1427 5661 29788 1427 387 2299 372 22 1586 1025 218 165 218 27 7092 22 266 25817 4550 2136 11 60847 9156 1077 797 27 689 3447 11146 108 1667 46829 161702 1113017 548478 24274 5062 46829']


In [23]:
#查看txt文件的前两行，观察是否正常
#用户可以指定分隔符，以应对不同分隔符的文件
def test_txt(file,separation_character = ','):
    with open(file, 'r') as f:
        line1 = f.readline().strip('\n').split(separation_character)
        line2 = f.readline().strip('\n').split(separation_character)
        print(line1)
        print(line2)
        print('长度：'+str(len(line1)))
    return True

#调用该函数
test_txt('title_tfidf_common_feature.txt',separation_character=',')

['0.25170819259181193', '0.2957471845505276', '0.06377310615759955', '0.0033096524488300002', '3.775622888877179', '0.2672547794653493']
['0.2573372917796861', '0.3457951561114256', '0.1283356866190581', '0.00520608968806812', '3.6027220849156056', '0.28216545286146155']
长度：6


True

In [ ]:
#text读出csv，传入text.txt，传出同样内容的csv文件
def text_to_csv(file,out_file,length = 1000000):
    #newline=''防止写csv时每两行数据中空一行
    with open(out_file, 'w',newline = '') as output:
        with open(file) as f:
            #处理开始时间
            start_time = time.time()
            #打开csv_reader以及csv_writer
            writer = csv.writer(output)
            
            for i in tqdm(range(length),mininterval=1.0):
                line = f.readline().strip('\n\t')
                line = line.split(',')
                #写下csv一行
                writer.writerow([line[0],line[1],line[2]])
    print("complete, "+"using time:"+str(time.time()-start_time))

text_to_csv('qauc_true.txt','qauc_true.csv')

In [9]:
#获得train.csv中的query和title长度以及它们的平均值，对后面模型的建立有指导作用
def evaluate_len(data_file,length):
    with open(data_file) as train_data:
        train_reader = csv.reader(train_data, delimiter=',')
        query_id_set = set()
        query_info=[0,10000,0]#max,min,avg
        title_info=[0,10000,0]
        #流式算法，如果有比最大值大的，取代原最大值，最小值同理
        for row in tqdm(train_reader):
            if len(row[1].split(' '))>query_info[0]:
                query_info[0] = len(row[1].split(' '))
            if len(row[3].split(' '))>title_info[0]:
                title_info[0] = len(row[3].split(' '))
            if len(row[1].split(' '))<query_info[1]:
                query_info[1] = len(row[1].split(' '))
            if len(row[3].split(' '))<title_info[1]:
                title_info[1] = len(row[3].split(' '))
            #平均值的计算（先除再加）
            query_info[2] += len(row[1].split(' '))/length
            title_info[2] += len(row[3].split(' '))/length
        print('max '+str(query_info[0])+","+str(title_info[0])+' min '+str(query_info[1])+","+str(title_info[1])+' avg '+str(query_info[2])+","+str(title_info[2]))
    return True

#调用该函数
evaluate_len('train.csv',20000)

20000it [00:00, 226591.14it/s]


max 300,35 min 1,1 avg 4.026250000000656,13.699550000000146


True